In [1]:
import math
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)

## 独热码转换

In [2]:
t = F.one_hot(torch.tensor([0, 2]), len(vocab))

t, t.shape

(tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0],
         [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0]]),
 torch.Size([2, 28]))

minibatch 数据的形状为： `(batch 大小, 时间步数)`，`one_hot` 函数可以将这样的数据转为三维张量。

In [3]:
x = torch.arange(10).reshape(2, 5)
x

tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])

In [4]:
x_processed = F.one_hot(x, 10)
x_processed, x_processed.shape

(tensor([[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]],
 
         [[0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]]),
 torch.Size([2, 5, 10]))

## 模型参数的初始化

In [5]:
def get_params(vocab_size, num_hiddens, device):
    """
    初始化 RNN 的参数
    """
    num_inputs = num_outputs = vocab_size

    def normal(shape):
        return torch.randn(size=shape, device=device) * 0.01

    # 隐藏层参数
    W_xh = normal((num_inputs, num_hiddens))
    W_hh = normal((num_hiddens, num_hiddens))
    b_h = torch.zeros(num_hiddens, device=device)
    # 输出层参数
    W_hq = normal((num_hiddens, num_outputs))
    b_q = torch.zeros(num_outputs, device=device)
    # 附加梯度
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.requires_grad_(True)
    return params


初始化隐状态：隐状态初始的时候可以全用 0 填充。

隐状态的初始形状为：`(batch_size, num_hiddens)`

注意，这是训练时的隐状态。我们的训练目标是传递的 **权重矩阵**，而不是训练状态本身

In [6]:
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), ) # 这里使用元组，和后面的构造对应

训练过程的前向计算函数：在一个时间步上计算下一个隐状态和输出。

In [7]:
def rnn(inputs, state, params):
    # inputs 的形状: (time_steps, batch_size, vocab_size)
    W_xh, W_hh, b_h, W_hq, b_q = params
    h, = state
    outputs = []

    # 向前计算 N 步
    # x 的形状: (batch_size, vocab_size)
    for x in inputs:
        h = torch.tanh(x @ W_xh + h @ W_hh + b_h)
        y = h @ W_hq + b_q
        outputs.append(y)
    return torch.cat(outputs, dim=0), (h, )

In [8]:
class RNNModel:
    def __init__(self, vocab_size: int, num_hiddens: int, device: torch.device,
        get_params, init_state, forward_fn):
        self.vocab_size, self.num_hiddens = vocab_size, num_hiddens
        self.params = get_params(vocab_size, num_hiddens, device)
        self.init_state, self.forward_fn = init_state, forward_fn
    
    def __call__(self, x: torch.Tensor, state):
        x = F.one_hot(x.T, self.vocab_size).type(torch.float32)
        return self.forward_fn(x, state, self.params)
    
    def begin_state(self, batch_size, device):
        return self.init_state(batch_size, self.num_hiddens, device)

In [9]:
num_hiddens = 512
gpu = torch.device('cuda:0')
net = RNNModel(len(vocab), num_hiddens, gpu, get_params, init_rnn_state, rnn)
state = net.begin_state(x.shape[0], gpu)
y, new_state = net(x.to(gpu), state)
y.shape, len(new_state), new_state[0].shape

(torch.Size([10, 28]), 1, torch.Size([2, 512]))

## 定义预测函数

预测方法：用户先给定一个 `prefix`，这个过程将隐状态不断更新，但是不产生输出。这个过程称为 warm-up。

Warm-up 之后，开始预测输出。

In [10]:
def predict_ch8(prefix: str, num_preds: int, net, vocab, device):
    """
    预测 prefix 后的输出
    """
    state = net.begin_state(batch_size=1, device=device) # 预测时，要求 batch 大小为 1
    outputs = [vocab[prefix[0]]]
    # 取上一个输出作为新的输入
    get_input = lambda: torch.tensor([outputs[-1]], device=device).reshape(1, 1)
    for y in prefix[1:]: # warm-up
        _, state = net(get_input(), state)
        outputs.append(vocab[y])
    for _ in range(num_preds): # 正式预测
        y, state = net(get_input(), state)
        outputs.append(int(y.argmax(dim=1).reshape(1)))
    return ''.join([vocab.idx_to_token[i] for i in outputs])

In [11]:
def grad_clipping(net, theta):  #@save
    """裁剪梯度"""
    if isinstance(net, nn.Module):
        params = [p for p in net.parameters() if p.requires_grad]
    else:
        params = net.params
    norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm


In [12]:
def train_epoch_ch8(net, train_iter, loss, updater, device, use_random_iter: bool):
    """
    一个 epoch 下的网络训练
    """
    state = None # 每个 epoch 都重置隐状态，但不重置权重矩阵
    loss_sum, num_train_tokens = 0, 0
    for x, y in train_iter:
        if state is None or use_random_iter:
            state = net.begin_state(batch_size=x.shape[0], device=device)
        else:
            # state 可能为张量，也可能为包含张量的元组
            if isinstance(net, nn.Module) and not isinstance(state, tuple):
                state.detach_()
            else:
                for s in state:
                    s.detach_()
        y = y.T.reshape(-1)
        x, y = x.to(device), y.to(device)
        y_hat, state = net(x, state) # 进行一步迭代
        l = loss(y_hat, y.long()).mean()
        if isinstance(updater, torch.optim.Optimizer):
            updater.zero_grad()
            l.backward()
            grad_clipping(net, 1)
            updater.step()
        else:
            l.backward()
            grad_clipping(net, 1)
            updater(batch_size=1)
        loss_sum += l * y.numel()
        num_train_tokens += y.numel()
    return math.exp(loss_sum / num_train_tokens), None

In [13]:
def train_ch8(net, train_iter, vocab, lr, num_epochs, device, use_random_iter=False):

    loss = nn.CrossEntropyLoss()
    if(isinstance(net, nn.Module)):
        updater = torch.optim.SGD(net.parameters(), lr)
    else:
        updater = lambda batch_size: d2l.sgd(net.params, lr, batch_size)
    predict = lambda prefix: predict_ch8(prefix, 50, net, vocab, device)

    for epoch in range(num_epochs):
        ppl, _ = train_epoch_ch8(net, train_iter, loss, updater, device, use_random_iter)
        print(f"epoch {epoch + 1}, perplexity: {ppl}")
        print(predict('time traveller')) # 写死了 predict 预测的内容
    print(f"Perplexity: {ppl}.")
    print(predict('time traveller'))

In [14]:
num_epochs, lr = 500, 1
train_ch8(net, train_iter, vocab, lr, num_epochs, d2l.try_gpu())

epoch 1, perplexity: 24.775164704608677
time traveller                                                  
epoch 2, perplexity: 19.756458558140054
time traveller                                                  
epoch 3, perplexity: 17.827680346356367
time travellereeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee
epoch 4, perplexity: 17.568218274352695
time travellereetttttttttttttttttttttttttttttttttttttttttttttttt
epoch 5, perplexity: 17.118367763011655
time traveller                                                  
epoch 6, perplexity: 16.66585279326261
time traveller                                                  
epoch 7, perplexity: 16.11240023030183
time travellere te te te te te te te te te te te te te te te te 
epoch 8, perplexity: 15.414320264468753
time traveller t t t t t t t t t t t t t t t t t t t t t t t t t
epoch 9, perplexity: 14.49496953392066
time traveller the the the the the the the the the the the the t
epoch 10, perplexity: 13.705168932647213
time traveller th